In [11]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
from mvl2 import genData2 as genData
from torch import tensor

liabParams55cov = genData.genParams(pis=tensor([.1, .1, .05]), rrMeans=tensor([5., 2.]), afMean = tensor(1e-4), pDs = tensor([.01, .01]), afShape=tensor(50.), nCases=tensor([1.5e4, 1.5e4, 4e3]), nCtrls=tensor(5e4), covShared=tensor([ [1, .95], [.95, 1] ]), meanEffectCovarianceScale=tensor(.01))[0]
liabParams55cov["pDs"] = liabParams55cov["pDs"][0:2]
liabData55cov = genData.v6liability(**liabParams55cov)
liabParams55cov["pDs"] = liabData55cov["PDs"]


PD1 threshold, PD2 threshold tensor(-2.3263) tensor(-2.3263)
pDsWithBoth tensor([0.0100, 0.0100, 0.0067], dtype=torch.float64)
pdThresh tensor([2.3263, 2.3263])
pdTarget tensor([0.0500, 0.0200])
pdvthresh tensor([1.6449, 2.0537])
meanEffect tensor([0.6815, 0.2726])
meanEffectsAcrossAllGenes tensor([0.6815, 0.2726])
allEffects tensor([[-0.6384, -0.2505],
        [-0.7016, -0.2790],
        [-0.6349, -0.2090],
        ...,
        [-0.7278, -0.3194],
        [-0.7060, -0.3028],
        [-0.5569, -0.1724]])
allEffects[i] tensor([-0.6384, -0.2505])
PDBothGivenV.mean tensor(0.0195, dtype=torch.float64)
PDBothGivenV / PDBoth tensor(2.9063, dtype=torch.float64)
pdsCovarOnMean.mean(0) tensor([0.0510, 0.0205, 0.0195], dtype=torch.float64)
np.corrcoef(pdvInBoth[:,0], pdvInBoth[:,1])
 [[1.         0.94889423]
 [0.94889423 1.        ]]
np.corrcoef(pdvInBoth[:,0], pdvInBoth[:,2])
 [[1.         0.96087044]
 [0.96087044 1.        ]]
PDBoth1GivenV tensor([0.0266, 0.0407, 0.0272,  ..., 0.0363, 0.0214, 

In [17]:
import numpyro
from numpyro.distributions import Multinomial, Beta, Dirichlet, DirichletMultinomial, Gamma, Beta, Categorical, Uniform, MultivariateNormal, Normal, LogNormal, Exponential, HalfCauchy, LKJCholesky
from jax import random
from numpyro.infer import MCMC, NUTS, SA, Predictive
import numpy as np
import jax.numpy as jnp
import jax

numpyro.set_host_device_count(6)
numpyro.enable_x64()


def get_pdhat(params):
    nCases = params["nCases"].numpy()
    nCtrls = params["nCtrls"].numpy()
    samplePDs = nCases / (nCases.sum() + nCtrls)
    pdsAll = np.array([1 - samplePDs.sum(), *samplePDs])
    return pdsAll

# mu_exp, var_exp = get_log_params(liabParams55cov["afMean"].numpy(), 1)
# Expected number of components
# For 2 case types it's
# none, 1only, 2only, both
# For 3 it's
# none, 1only, 2only, 3only, 1&2, 1&3, 2&3, 123 (7)
# for 4 it's
# none, 1only, 2only, 3only, 4only, 1&2, 1&3, 1&4, 2&3, 2&4, 3&4, 123, 124, 134, 234, 1234
# which is 4 + 4choose2 + nchoose3  + nchoose4 
def mix_weights(beta: jnp.array):
    beta_cumprod = (1 - beta).cumprod(-1)
    return jnp.pad(beta, (0,1), constant_values=1) * jnp.pad(beta_cumprod, (1,0), constant_values = 1)

# Covariates needed
# Sex of the individual
# parent of origin would be important
def model(data, params, nHypotheses: int, alpha = .05):
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha / nHypotheses))

    with numpyro.plate("prob_plate", nHypotheses):
        pd_hat = get_pdhat(params)
        conc = numpyro.sample('conc', Exponential(pd_hat).to_event(1))
        probs = numpyro.sample("probs", Dirichlet(conc))

    with numpyro.plate("data", data.shape[0]):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

# WIP Probably not working yet
def model_mvn(data, params, nHypotheses: int, alpha = .05):
    kSampleCategories = data.shape[1]

    # If in pyro: conc = numpyro.sample('conc', Exponential(exponential_prior).to_event(1))
    with numpyro.plate("beta_plate", nHypotheses-1):
        beta = numpyro.sample("beta", Beta(1, alpha / nHypotheses))

    with numpyro.plate("prob_plate", nHypotheses):
        theta = numpyro.sample("theta", HalfCauchy(np.ones(kSampleCategories)))
        L_omega = numpyro.sample("L_omega", LKJCholesky(kSampleCategories, np.ones(1) ).to_event(1))
        c = jax.numpy.matmul(jax.numpy.diag(theta.sqrt()), L_omega)
        print("L_omega", L_omega.shape)
        # TODO: understand this note from Pyro code
        # For inference with SVI, one might prefer to use torch.bmm(theta.sqrt().diag_embed(), L_omega)

        # Vector of expectations
        probs = dist.MultivariateNormal(pdsAll, scale_tril=L_Omega)

    with numpyro.plate("data", data.shape[0]):
        z = numpyro.sample("z", Categorical(mix_weights(beta)))
        return numpyro.sample("obs", Multinomial(probs=probs[z]), obs=data)

def get_posterior_samples(mcmc):
    posterior_samples = mcmc.get_samples()
    posterior_samples['pi'] = stickbreak(posterior_samples['beta'])
    
    return posterior_samples

print("Ready")

Ready


In [18]:
mcmc_mvn = MCMC(NUTS(model_mvn, max_tree_depth=8), num_warmup=200, num_samples=1000)
mcmc_mvn.run(random.PRNGKey(12269), liabData55cov["altCounts"].numpy(), liabParams55cov, 12)
mcmc_mvn.print_summary()

ValueError: Incompatible shapes for broadcasting: ((12,), (4,))

In [8]:
mcmc = MCMC(NUTS(model, max_tree_depth=8), num_warmup=200, num_samples=1000)
mcmc.run(random.PRNGKey(12269), liabData55cov["altCounts"].numpy(), liabParams55cov, 12)
mcmc.print_summary()

warmup:   2%|▏         | 26/1200 [00:46<34:54,  1.78s/it, 255 steps of size 8.16e-03. acc. prob=0.70]


KeyboardInterrupt: 

In [ ]:
posterior_samples = mcmc.get_samples()

prob = posterior_samples["probs"]
print(prob.mean(0))

beta = posterior_samples['beta']

weights = mix_weights(beta)
print("inferred stick-breaking weights mean: ", weights.mean(0))
print("inferred stick-breaking weights stdd: ", weights.std(0))

